In [ ]:
import pandas as pd
import spacy
from book_importer import BookImporter


In [ ]:
book_id = [43, 345, 41445, 209, 1513]
book_data = []
for book in book_id:
    title, author, text = BookImporter.get_book(book)
    book_data.append({'Title': title, 'Author': author, 'Text': text})


In [ ]:
gothic_data = pd.DataFrame(book_data)
gothic_data.info()

In [ ]:
print(gothic_data)

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
jekyll_hyde_doc = nlp(jekyll_hyde)
jekyll_hyde_vectors = [token.vector for token in jekyll_hyde_doc]

In [ ]:
dracula_doc = nlp(dracula)
dracula_vectors = [token.vector for token in dracula_doc]

In [ ]:
frankenstein_doc = nlp(frankenstein)
frankenstein_vectors = [token.vector for token in frankenstein_doc]

In [ ]:
turn_of_the_screw_doc = nlp(turn_of_the_screw)
turn_of_the_screw_vectors = [token.vector for token in turn_of_the_screw_doc]

In [ ]:
romeo_and_juliet_doc = nlp(romeo_and_juliet)
romeo_and_juliet_vectors = [token.vector for token in romeo_and_juliet_doc]

In [ ]:
dracula_jekyll_similarity = dracula_doc.similarity(jekyll_hyde_doc)
print(dracula_jekyll_similarity)

In [ ]:
romeo_dracula_similarity = dracula_doc.similarity(romeo_and_juliet_doc)
print(romeo_dracula_similarity)